In [1]:
from pyspark.sql import SparkSession
import os

# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.12:0.16.0 pyspark-shell'

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
1,application_1674913086828_0002,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
%%sh
spark-submit --version

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /___/ .__/\_,_/_/ /_/\_\   version 3.3.0-amzn-1
      /_/
                        
Using Scala version 2.12.15, OpenJDK 64-Bit Server VM, 1.8.0_342
Branch 
Compiled by user release on 2022-11-02T06:24:47Z
Revision 
Url 
Type --help for more information.


In [3]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql import functions as f
from pyspark.sql import Window
from pyspark.sql.types import StringType, DoubleType, IntegerType

spark = SparkSession.builder \
                    .appName('PK_BigData_Project') \
                    .config('spark.jars.packages', 'com.amazonaws:aws-java-sdk-bundle:1.12.170') \
                    .config('spark.jars.packages', 'org.apache.hadoop:hadoop-aws:3.3.3') \
                    .config("spark.driver.memory", "12g") \
                    .config("spark.executor.memory", "12g") \
                    .master('local[3]') \
                    .getOrCreate()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
filePath = "s3a://pkruk-big-data/data/"

filenames = ['Badges', 'Comments', 'PostHistory', \
             'PostLinks', 'Posts', 'Tags', 'Users', \
             'Votes']

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
dataframes = {}

for name in filenames:
    dataframes[name] = spark.read.format('com.databricks.spark.xml').option('rootTag', name.lower()).option('rowTag', 'row').load(filePath + name + '.xml')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Zmiana nazw kolumn, sprawdzenie typów, czyszczenie wartości

### 1. Badges

In [6]:
dataframes['Badges'] = dataframes['Badges'] \
                        .withColumnRenamed('_Date', 'badge_aquire_date') \
                        .withColumnRenamed('_Id', 'badge_id') \
                        .withColumnRenamed('_Name', 'badge_name') \
                        .withColumnRenamed('_TagBased', 'is_badge_tag_based') \
                        .withColumnRenamed('_UserId', 'badge_owner_id') \
                        .withColumnRenamed('_Class', 'badge_class') \
                        .dropDuplicates()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
# dataframes['Badges'].printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
print(dataframes['Badges'].subtract(dataframes['Badges'].dropna()).count())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0

In [9]:
# nie ma null, typy są ok

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### 2. Comments

In [10]:
dataframes['Comments'] = dataframes['Comments'] \
                        .drop('_ContentLicense') \
                        .withColumnRenamed('_CreationDate', 'comment_creation_date') \
                        .withColumnRenamed('_Id', 'comment_id') \
                        .withColumnRenamed('_PostId', 'post_id') \
                        .withColumnRenamed('_Score', 'comment_score') \
                        .withColumnRenamed('_Text', 'comment_content') \
                        .withColumnRenamed('_UserId', 'user_id') \
                        .withColumnRenamed('_UserDisplayName', 'user_display_name') \
                        .dropDuplicates()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
# dataframes['Comments'].printSchema() typy sa ok

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
print(dataframes['Comments'].count())
print(dataframes['Comments'].subtract(dataframes['Comments'].dropna()).count())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

70647
70647

In [13]:
dataframes['Comments'].show(20)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------------+----------+-------+-------------+--------------------+-----------------+-------+
|comment_creation_date|comment_id|post_id|comment_score|     comment_content|user_display_name|user_id|
+---------------------+----------+-------+-------------+--------------------+-----------------+-------+
| 2013-11-08 12:11:...|      1024|    894|            1|Sorry for a littl...|             null|    115|
| 2013-11-17 01:01:...|      1114|    938|            0|@barrycarter *"Th...|             null|    276|
| 2013-12-30 17:51:...|      1826|   1274|            0|Mars is also -128...|             null|    538|
| 2014-04-07 11:32:...|      3446|   2115|            2|from what is writ...|             null|    992|
| 2014-07-11 07:27:...|      5901|   2489|            0|Which is why I ha...|             null|    864|
| 2014-08-11 15:25:...|      6284|   6083|            0|Really good answe...|             null|   2153|
| 2014-11-08 16:28:...|      7965|   7905|            2|Where di

In [14]:
## wszystkie user_name gdzie user_id jest nullem
# user_display_names_for_null_id = dataframes['Comments'].filter(dataframes['Comments'].user_id.isNull()).select(f.col('user_display_name')).distinct().withColumnRenamed('user_display_name', 'user_name')
# user_display_names_for_null_id.show(3)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
# teraz mam wszystkie user_display_name gdzie brakuje user_id i pasujące user_id
# user_display_name_and_id = user_display_names_for_null_id.join(dataframes['Users'], user_display_names_for_null_id.user_name == dataframes['Users'].user_display_name, 'left') \
#                                 .select('user_name', 'user_id') \
#                                 .withColumnRenamed('user_id', 'missing_user_id') \
#                                 .withColumnRenamed('user_name', 'missing_id_user_name')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
# dataframes_comments = dataframes['Comments'].join(user_display_name_and_id, user_display_name_and_id.missing_id_user_name == dataframes['Comments'].user_display_name, 'left') \
#                         .withColumn('user_id', f.coalesce(dataframes['Comments'].user_id, user_display_name_and_id.missing_user_id)) \
#                         .drop('user_display_name', 'missing_id_user_name', 'missing_user_id') \
#                         .dropDuplicates()
# dataframes_comments.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
all_comment_count = dataframes['Comments'].count()
comment_where_user_is_null = dataframes['Comments'].where(f.col('user_id').isNull()).count()

print(comment_where_user_is_null / all_comment_count * 100)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

3.414157713703342

In [18]:
dataframes['Comments'] = dataframes['Comments'] \
                            .drop('user_display_name') \
                            .dropna(subset='user_id')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
all_comment_count = dataframes['Comments'].count()
comment_where_user_is_null = dataframes['Comments'].where(f.col('user_id').isNull()).count()

print(comment_where_user_is_null / all_comment_count * 100)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0.0

### 3. PostHistory

In [20]:
dataframes['PostHistory'] = dataframes['PostHistory'] \
                        .drop('_UserDisplayName', '_ContentLicense') \
                        .withColumnRenamed('_Comment', 'comment_content') \
                        .withColumnRenamed('_CreationDate', 'post_history_creation_date') \
                        .withColumnRenamed('_Id', 'post_history_id') \
                        .withColumnRenamed('_PostHistoryTypeId', 'post_history_type_id') \
                        .withColumnRenamed('_PostId', 'related_post_id') \
                        .withColumnRenamed('_RevisionGUID', 'revision_guid') \
                        .withColumnRenamed('_Text', 'post_history_text') \
                        .withColumnRenamed('_UserId', 'post_history_user_id') \
                        .dropDuplicates()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
# dataframes['PostHistory'].printSchema() typu danych sa ok

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
print(dataframes['PostHistory'].count())
print(dataframes['PostHistory'].subtract(dataframes['PostHistory'].dropna()).count())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

106074
73837

In [23]:
dataframes['PostHistory'].show(20)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------------+---------------+--------------------+---------------+--------------------+--------------------+--------------------+
|     comment_content|post_history_creation_date|post_history_id|post_history_type_id|related_post_id|       revision_guid|   post_history_text|post_history_user_id|
+--------------------+--------------------------+---------------+--------------------+---------------+--------------------+--------------------+--------------------+
|added 4 character...|      2013-09-25 12:56:...|            325|                   5|            122|7363fe22-a136-4f6...|I want to know th...|                  99|
|Proposed by 19 ap...|      2013-09-27 14:14:...|            859|                  24|            290|727eb623-33b1-4d5...|                null|                null|
|                null|      2013-10-03 08:36:...|           1343|                   2|            482|64f25b33-7497-4d1...|What you describe...|                  13|
|Add

In [24]:
all_post_history_row_count = dataframes['PostHistory'].count()
post_history_rows_where_user_is_null = dataframes['PostHistory'].where(f.col('post_history_user_id').isNull()).count()

print(post_history_rows_where_user_is_null / all_post_history_row_count * 100)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

12.68077002847069

### 4. PostLinks

In [25]:
dataframes['PostLinks'] = dataframes['PostLinks'] \
                        .withColumnRenamed('_CreationDate', 'post_creation_date') \
                        .withColumnRenamed('_Id', 'post_links_id') \
                        .withColumnRenamed('_LinkTypeId', 'link_type_id') \
                        .withColumnRenamed('_PostId', 'post_id') \
                        .withColumnRenamed('_RelatedPostId', 'related_post_id') \
                        .dropDuplicates()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [26]:
# dataframes['PostLinks'].printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [27]:
print(dataframes['PostLinks'].count())
print(dataframes['PostLinks'].subtract(dataframes['PostLinks'].dropna()).count())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

6523
0

In [28]:
dataframes['PostLinks'].show(20)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-------------+------------+-------+---------------+
|  post_creation_date|post_links_id|link_type_id|post_id|related_post_id|
+--------------------+-------------+------------+-------+---------------+
|2013-10-29 09:07:...|         1690|           1|    815|             24|
|2015-10-08 14:01:...|        65312|           1|  12052|          11753|
|2016-10-05 20:14:...|       113358|           1|  18592|           1801|
|2017-08-18 04:00:...|       159741|           3|  22139|           8117|
|2017-10-30 13:20:...|       184914|           3|  23355|          15026|
|2019-04-12 08:38:...|       315502|           3|  30391|          30313|
|2020-06-28 18:12:...|       401643|           1|  36719|          21284|
|2021-08-04 02:45:...|       504636|           1|  45138|          44851|
|2019-03-25 00:40:...|       311180|           1|  30096|          30110|
|2019-12-04 06:58:...|       367704|           1|  34155|          28314|
|2020-01-07 01:01:...|       371580|  

### 5. Tags

In [29]:
# Tags

dataframes['Tags'] = dataframes['Tags'] \
                        .withColumnRenamed('_Count', 'tag_count') \
                        .withColumnRenamed('_ExcerptPostId', 'tag_excerpt_post_id') \
                        .withColumnRenamed('_Id', 'tag_id') \
                        .withColumnRenamed('_TagName', 'tag_name') \
                        .withColumnRenamed('_WikiPostId', 'tag_wiki_post_id') \
                        .dropDuplicates()

# dataframes['Tags'].printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [30]:
print(dataframes['Tags'].count())
print(dataframes['Tags'].subtract(dataframes['Tags'].dropna()).count())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

570
179

In [31]:
dataframes['Tags'].show(20)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+-------------------+------+--------------------+----------------+
|tag_count|tag_excerpt_post_id|tag_id|            tag_name|tag_wiki_post_id|
+---------+-------------------+------+--------------------+----------------+
|       20|               null|   658|         occultation|            null|
|       12|              11221|   360|  impact-probability|           11220|
|       29|              41639|   234|cosmological-horizon|           41638|
|        3|               null|  1042|    v838-monocerotis|            null|
|        9|              41439|   603|           hipparcos|           41438|
|        5|               null|  1105|              simbad|            null|
|        9|               null|   378|                disk|            null|
|       38|              41502|   441|              crater|           41501|
|       29|              41678|    92|             azimuth|           41677|
|        3|              41332|   825|             nemesis|           41331|

### 6. Users

In [32]:
dataframes['Users'] = dataframes['Users'] \
                        .drop('_WebsiteUrl', '_ProfileImageUrl', '_Location', '_AboutMe') \
                        .withColumnRenamed('_AccountId', 'account_id') \
                        .withColumnRenamed('_CreationDate', 'user_creation_date') \
                        .withColumnRenamed('_DisplayName', 'user_display_name') \
                        .withColumnRenamed('_LastAccessDate', 'user_last_access_date') \
                        .withColumnRenamed('_Reputation', 'user_reputation') \
                        .withColumnRenamed('_UpVotes', 'user_upvotes') \
                        .withColumnRenamed('_Id', 'user_id') \
                        .withColumnRenamed('_DownVotes', 'user_downvotes') \
                        .withColumnRenamed('_Views', 'user_views') \
                        .dropDuplicates()

# dataframes['Users'].printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [33]:
print(dataframes['Users'].count())
print(dataframes['Users'].dropna().count())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

34611
34611

In [34]:
dataframes['Users'].show(20)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+--------------------+-----------------+--------------+-------+---------------------+---------------+------------+----------+
|account_id|  user_creation_date|user_display_name|user_downvotes|user_id|user_last_access_date|user_reputation|user_upvotes|user_views|
+----------+--------------------+-----------------+--------------+-------+---------------------+---------------+------------+----------+
|    320233|2013-09-24 19:29:...|              Rob|             0|     41| 2021-01-16 08:24:...|            839|           5|        16|
|    118337|2013-09-25 19:03:...|   Michael Mrozek|             0|    110| 2017-07-28 18:39:...|            101|           0|         0|
|    204168|2013-12-11 05:10:...|        javabrett|             0|    575| 2021-07-03 01:31:...|            101|           0|         0|
|     42183|2014-05-11 12:21:...|            Frank|             0|   1640| 2014-05-11 12:21:...|            101|           1|         0|
|   3180787|2014-09-06 21:52:...|      dg

### 7. Votes

In [35]:
dataframes['Votes'] = dataframes['Votes'] \
                        .withColumnRenamed('_CreationDate', 'vote_creation_date') \
                        .withColumnRenamed('_Id', 'vote_id') \
                        .withColumnRenamed('_PostId', 'voted_post_id') \
                        .withColumnRenamed('_UserId', 'user_voter_id') \
                        .withColumnRenamed('_VoteTypeId', 'vote_type_id') \
                        .withColumnRenamed('_BountyAmount', 'count_bounty_amount') \
                        .dropDuplicates()

# dataframes['Votes'].printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [36]:
print(dataframes['Votes'].count())
print(dataframes['Votes'].subtract(dataframes['Votes'].dropna()).count())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

210062
209591

In [37]:
dataframes['Votes'].show(20)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+-------------------+-------+-------------+-------------+------------+
|count_bounty_amount| vote_creation_date|vote_id|voted_post_id|user_voter_id|vote_type_id|
+-------------------+-------------------+-------+-------------+-------------+------------+
|               null|2013-09-26 00:00:00|    783|           94|         null|           2|
|               null|2013-10-04 00:00:00|   2293|          489|         null|           2|
|               null|2013-12-04 00:00:00|   6701|         1054|         null|           2|
|               null|2014-02-19 00:00:00|  10030|         1465|         null|           2|
|               null|2014-03-11 00:00:00|  10502|         1862|         null|           2|
|               null|2014-03-18 00:00:00|  10679|         1999|         null|           2|
|               null|2014-04-22 00:00:00|  11705|         2222|         null|           2|
|               null|2014-04-30 00:00:00|  12000|         2319|         null|           2|

## 8. Posts

In [38]:
dataframes['Posts'] = dataframes['Posts'] \
                        .drop('_ContentLicense', '_LastEditorDisplayName', '_OwnerDisplayName') \
                        .withColumnRenamed('_AcceptedAnswerId', 'accepted_answer_id') \
                        .withColumnRenamed('_AnswerCount', 'post_answer_count') \
                        .withColumnRenamed('_Body', 'post_body') \
                        .withColumnRenamed('_ClosedDate', 'post_closed_date') \
                        .withColumnRenamed('_CommentCount', 'post_comment_count') \
                        .withColumnRenamed('_CommunityOwnedDate', 'post_community_owned_date') \
                        .withColumnRenamed('_CreationDate', 'post_creation_date') \
                        .withColumnRenamed('_FavoriteCount', 'post_favourite_count') \
                        .withColumnRenamed('_Id', 'post_id') \
                        .withColumnRenamed('_LastActivityDate', 'post_last_activity_date') \
                        .withColumnRenamed('_LastEditDate', 'post_last_edited_date') \
                        .withColumnRenamed('_LastEditorUserId', 'post_last_editor_user_id') \
                        .withColumnRenamed('_OwnerUserId', 'post_owner_user_id') \
                        .withColumnRenamed('_ParentId', 'post_parent_id') \
                        .withColumnRenamed('_PostTypeId', 'post_type_id') \
                        .withColumnRenamed('_Score', 'post_score') \
                        .withColumnRenamed('_Title', 'post_title') \
                        .withColumnRenamed('_ViewCount', 'post_view_count') \
                        .withColumnRenamed('_Tags', 'post_tags') \
                        .dropDuplicates()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [41]:
for name in filenames:
    dataframes[name].write.parquet('s3://pkruk-big-data/processed/' + name + '.parquet')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…